# california-coronavirus-data examples

By [Ben Welsh](https://palewi.re/who-is-ben-welsh)

A demonstration of how to use Python to work with the Los Angeles Times' independent tally of coronavirus cases in California published on GitHub at [datadesk/california-coronavirus-data](https://github.com/datadesk/california-coronavirus-data#state-cdph-totalscsv). To run this notebook immediately in the cloud,  click the [Binder](https://mybinder.org/) launcher below.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/datadesk/california-coronavirus-data/master?urlpath=lab/tree/notebooks/examples.ipynb)

## Import Python tools

Our data analysis and plotting tools

In [1]:
import pandas as pd
import altair as alt

Customizations to the Altair theme

In [2]:
import altair_latimes as lat

In [3]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

## Import data

Read in the agency totals

In [4]:
agency_df = pd.read_csv(
    "../latimes-agency-totals.csv",
    parse_dates=["date"]
)

In [5]:
agency_df.head()

,agency,county,fips,date,confirmed_cases,deaths,did_not_update
0,Alameda,Alameda,1,2020-04-12,807,22,NaN
1,Berkeley,Alameda,1,2020-04-12,38,1,True
2,Alpine,Alpine,3,2020-04-12,1,0,True
3,Amador,Amador,5,2020-04-12,7,0,True
4,Butte,Butte,7,2020-04-12,14,0,True


In [6]:
agency_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2086 entries, 0 to 2085
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   agency           2086 non-null   object        
 1   county           2086 non-null   object        
 2   fips             2086 non-null   int64         
 3   date             2086 non-null   datetime64[ns]
 4   confirmed_cases  2086 non-null   int64         
 5   deaths           2086 non-null   int64         
 6   did_not_update   160 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 114.2+ KB


## Aggregate data

### By state

Lump all the agencies together and you get the statewide totals.

In [7]:
state_df = agency_df.groupby(["date"]).agg({
    "confirmed_cases": "sum",
    "deaths": "sum"
}).reset_index()

In [8]:
state_df.head()

,date,confirmed_cases,deaths
0,2020-01-26,2,0
1,2020-01-27,2,0
2,2020-01-28,2,0
3,2020-01-29,2,0
4,2020-01-30,2,0


In [9]:
state_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             78 non-null     datetime64[ns]
 1   confirmed_cases  78 non-null     int64         
 2   deaths           78 non-null     int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 2.0 KB


### By county

Three cities &mdash; Berkeley, Long Beach and Pasadena &mdash; run independent public health departments. Calculating county-level totals requires grouping them with their local peers.

In [10]:
county_df = agency_df.groupby(["date", "county"]).agg({
    "confirmed_cases": "sum",
    "deaths": "sum"
}).reset_index()

In [11]:
county_df.head()

,date,county,confirmed_cases,deaths
0,2020-01-26,Alameda,0,0
1,2020-01-26,Calaveras,0,0
2,2020-01-26,Contra Costa,0,0
3,2020-01-26,Humboldt,0,0
4,2020-01-26,Los Angeles,1,0


In [12]:
county_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1951 entries, 0 to 1950
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             1951 non-null   datetime64[ns]
 1   county           1951 non-null   object        
 2   confirmed_cases  1951 non-null   int64         
 3   deaths           1951 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 61.1+ KB


## Chart the statewide totals over time

In [13]:
# Create a base chart with the common x-axis
chart = alt.Chart(state_df).encode(
    x=alt.X("date:T", title=None)
)

# Create the cases line
cases = chart.mark_line(color=lat.palette['default']).encode(
    y=alt.Y("confirmed_cases:Q", title="Confirmed cases")
)

# Create the deaths line
deaths = chart.mark_line(color=lat.palette['schemes']['ice-7'][3]).encode(
    y=alt.Y("deaths:Q", title="Deaths")
)

# Combine them into a single chart
(cases & deaths).properties(title="Statewide cumulative totals")

alt.VConcatChart(...)

## Chart the county totals

First on a linear scale

In [14]:
# Create the base chart
chart = alt.Chart(county_df).mark_line().encode(
    x=alt.X("date:T", title=None),
    color=alt.Color("county:N", title="County", legend=None)
)

# The cases line
cases = chart.encode(
    y=alt.Y(
        "confirmed_cases:Q",
        title="Confirmed cases"
    ),
)

# The deaths line
deaths = chart.mark_line().encode(
    y=alt.Y("deaths:Q", title="Deaths"),
)

# Combined into a chart
(cases & deaths).properties(title="Cumulative totals by county")

alt.VConcatChart(...)

Again on a logarithmic scale

In [15]:
# Make a base chart
chart = alt.Chart(county_df).mark_line().encode(
    x=alt.X("date:T", title=None),
    color=alt.Color("county:N", title="County", legend=None)
)

# The cases lines
cases = chart.transform_filter(alt.datum.confirmed_cases > 0).encode(
    y=alt.Y(
        "confirmed_cases:Q",
        scale=alt.Scale(type='log'),
        title="Confirmed cases"
    ),
)

# The deaths lines
deaths = chart.transform_filter(alt.datum.deaths > 0).encode(
    y=alt.Y(
        "deaths:Q",
        scale=alt.Scale(type='log'),
        title="Deaths"
    ),
)

# Slapping them together
(cases & deaths).properties(title="Cumulative totals by county")

alt.VConcatChart(...)

A common technique for clarifying these charts to begin each line on the day the county hit a minimum number. Let's try it with 10.

In [16]:
day_10_df = county_df[
    # Filter down to only days with 10 or more cumulative cases
    county_df.confirmed_cases >= 10
].groupby(
    # And then get the minimum date for each county
    'county'
).date.min().reset_index()

Merge that date to each row in the data.

In [17]:
county_date_diff_df = county_df.merge(
    day_10_df,
    how='inner',
    on='county',
    suffixes=['', '_gte_10_cases']
)

Calculate each day's distance from its tenth day.

In [18]:
county_date_diff_df['days_since_10'] = (county_date_diff_df.date - county_date_diff_df.date_gte_10_cases).dt.days

Chart it.

In [19]:
alt.Chart(county_date_diff_df).transform_filter(
    # Only keep everything once they hit 10 cases
    alt.datum.days_since_10 >= 0
).mark_line().encode(
    x=alt.X(
        "days_since_10:O",
        title="Days since 10th case"
    ),
    y=alt.Y(
        "confirmed_cases:Q",
        scale=alt.Scale(type='log'),
        title="Confirmed cases"
    ),
    color=alt.Color("county:N", title="County", legend=None)
).properties(title="Cumulative totals by county")

alt.Chart(...)

## Chart new cases and deaths

Calculate the number of new cases each day using panda's [diff](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.diff.html) method.

In [20]:
state_df['new_confirmed_cases'] = state_df.confirmed_cases.diff()

Do the same for deaths

In [21]:
state_df['new_deaths'] = state_df.deaths.diff()

Now calculate the moving seven-day average of each using panda's [rolling](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) method.

In [22]:
state_df['new_confirmed_cases_rolling_average'] = state_df.new_confirmed_cases.rolling(7).mean()

In [23]:
state_df['new_deaths_rolling_average'] = state_df.new_deaths.rolling(7).mean()

Put it all together on the chart 

In [24]:
# One base chart object with the data they all share
chart = alt.Chart(state_df).encode(
    x=alt.X("date:T", title=None),
)

# The new cases bars
cases_bars = chart.mark_bar(color=lat.palette['default']).encode(
    y=alt.Y(
        "new_confirmed_cases:Q",
        title="New confirmed cases"
    )
)

# The cases rolling average
cases_line = chart.mark_line(color=lat.palette['accent']).encode(
    y=alt.Y(
        "new_confirmed_cases_rolling_average:Q",
        title="7-day average"
    )
)

# The new deaths bars
deaths_bars = chart.mark_bar(color=lat.palette['schemes']['ice-7'][3]).encode(
    y=alt.Y(
        "new_deaths:Q",
        title="New deaths"
    )
)

# The deaths rolling average
deaths_line = chart.mark_line(color=lat.palette['schemes']['ice-7'][6]).encode(
    y=alt.Y(
        "new_deaths_rolling_average:Q",
        title="7-day average"
    )
)

# Combine it all together into one paired chart
((cases_bars + cases_line) & (deaths_bars + deaths_line)).properties(
    title="New case and deaths statewide by day"
)

alt.VConcatChart(...)